### Import Libraries

In [ ]:
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import dask.array as da
import s3fs
import dask
from dask.distributed import Client, LocalCluster

### Using DASK for parallel processing

Initialise a Dask Cluster and client for processing your Zarr Dataset, the dashboard link allows for simple visualisatoion of your ongoing processing.

In [ ]:
# Create a Dask cluster
cluster = LocalCluster()
client = Client(cluster)
client

### Access and Open the dataset and inspect using Xarray and Dask

Dask has integrated support for Xarray, as such by specifying the 'chunks' parameter when opening the dataset, the array will automatically be opened as a chunked dask array.

As such, the subsetting steps below will automatically processed the different chunks in parallel.

For additional information and implementation examples of DASK please refer to the library documentation: https://docs.dask.org/en/stable/

For specific examples of using Dask for geoprocessing aditionnal information can be found on Pange: https://gallery.pangeo.io/repos/pangeo-data/pangeo-tutorial-gallery/dask.html

In [ ]:
# Replace with your online zarr storage for your personal zarr files
host = 'https://s3.waw3-1.cloudferro.com'
bucket = 'emodnet'
arco_location = 'bathymetry/bathymetry_2022.zarr'

# Access the Dataset using xarray and dask
dataset = xr.open_dataset(f"{host}/{bucket}/{arco_location}", chunks='auto') #
dataset

### Defining regions of interest for subsetting the Bathymetry dataset.

In [ ]:
# Define bounding boxes, vmin, and vmax for different regions within European waters
regions = ['North Sea', 'Eastern Baltic Sea', 'Western Baltic Sea', 'Bay of Biscay', 'Brittany', 'Mediterranean Sea']       # Insert region name
bboxes = [(-2, 8, 50, 60), (17, 30, 55, 62), (7, 18, 53, 60), (-6.5, -0.5, 43,48), (-5.5, -1, 47, 50), (-6, 37, 30, 45.5)]  # Insert BBOX as Follows: [Wlat, Elat, Slon, Nlon) all in degrees.
vmins = [-500, -300, -500, -5000, -300, -5000]                                                                              # insert the max and min boundaries of the plotted colorbar
vmaxes = [0, 0, 0, 0, 0, 0]

### Define the plotting function using ax.imshow 
Using imshow for plotting the dataset has proven to run much faster with DASK than other conventional libraries such as matplotlib and cartopy.

In [ ]:
# Define colors for elevation levels
cmap = 'viridis'

# Function to plot a single chunk of elevation data with actual latitudes and longitudes
def plot_regions(chunk, ax, lat, lon, region, vmin, vmax):
    
    im = ax.imshow(chunk, cmap=cmap,interpolation='nearest', origin='lower', extent=(lon[0], lon[-1], lat[0], lat[-1]), vmin=vmin, vmax=vmax)
    
    lon_ticks = lon.values[::int(lon.size / 7)]  
    lat_ticks = lat.values[::int(lat.size / 7)]  
    
    ax.set_xticks(lon_ticks)  
    ax.set_yticks(lat_ticks)   
    ax.set_xlabel('Longitude (degrees)')
    ax.set_ylabel('Latitude (degrees)')
    ax.set_title(f'Elevation Map - {region}')
    ax.grid(visible=True, which='major', axis='both', color='black', linestyle='-', linewidth=0.5)

    ax.coastlines(resolution='10m')
    cbar = plt.colorbar(im, ax=ax, orientation='vertical',  fraction = 0.025)
    cbar.set_label('Elevation (m)')
   

### Subsetting the dataset and plotting



In [ ]:
%%time
# Loop through each region
for idx, (region, bbox, vmin, vmax) in enumerate(zip(regions, bboxes, vmins, vmaxes)):
    subset_dataset = dataset.sel(latitude=slice(bbox[2], bbox[3]), longitude=slice(bbox[0], bbox[1]))
    dask_array = subset_dataset['elevation']
    print(dask_array)
    
    # Create a figure for each region
    fig, ax = plt.subplots(1, 1, figsize=(8, 8), subplot_kw={'projection': ccrs.PlateCarree()})
    region_to_plot = dask_array.compute()                                                                       # Call .compute() to process the dataset using the DASK Cluster.
    plot_regions(region_to_plot, ax, region_to_plot.latitude, region_to_plot.longitude, region, vmin, vmax)
    
    plt.tight_layout()
    #plt.savefig(f"{region}_elevation_map.png")                                                                 # Uncomment and specify name and location to save each figure with region name
    plt.show()